# kryptonite

## Import modules & get data

In [1]:
from statsbombpy import sb
%matplotlib inline
import json
from pandas.io.json import json_normalize
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, ConnectionPatch
from matplotlib.offsetbox import  OffsetImage
import matplotlib.patches as mpatches
from functools import reduce
import plotly.graph_objects as px
import plotly.express as px
import plotly.graph_objs as go
import warnings
from plotly.validators.scatter.marker import SymbolValidator
import chart_studio.plotly as py
import chart_studio.tools as tls
import chart_studio
import math
from plotly.subplots import make_subplots

In [3]:
%%capture --no-display
#get invincibles season open data from Statsbomb

#sb.competitions()
matches_0304_df = sb.matches(competition_id=2, season_id=44)

matchid_list = matches_0304_df["match_id"].tolist()

#print(matchid_list)

seasonevents_df = sb.events(match_id=3749052)
seasonevents_df = seasonevents_df[0:0] 
#print(seasonevents_df)

for i in matchid_list:
    events_df = sb.events(match_id=i)
    seasonevents_df = seasonevents_df.append(events_df)



In [4]:
arsenal_passes_df = seasonevents_df[(seasonevents_df.type == "Pass") & (seasonevents_df.team == "Arsenal")]
arsenal_shots_df = seasonevents_df[(seasonevents_df.type == "Shot") & (seasonevents_df.team == "Arsenal")]

In [5]:
arsenal_passes_df = arsenal_passes_df[arsenal_passes_df["pass_outcome"].isnull()]
#print(arsenal_passes_df)

## Matrices

### All Passes

In [43]:
ars_count_passes_df = arsenal_passes_df[['player', 'pass_recipient']]
ars_players = ["Ashley Cole", "Dennis Bergkamp", "Patrick Vieira", 'Thierry Henry', 'Sulzeer Jeremiah "Sol" Campbell', "Robert Pirès ", "Kolo Habib Touré", "Gilberto Aparecido da Silva", "Laureano Bisan-Etame Mayer", "Fredrik Ljungberg"]
ars_count_passes_df = ars_count_passes_df.value_counts().to_frame('Count').reset_index()
ars_count_passes_df = ars_count_passes_df[ars_count_passes_df['player'].isin(ars_players)]  
ars_count_passes_df = ars_count_passes_df[ars_count_passes_df['pass_recipient'].isin(ars_players)]  
#print(ars_count_passes_df)
ars_pivot_df = ars_count_passes_df.pivot("player", "pass_recipient", "Count")

In [44]:
fig = px.imshow(ars_pivot_df, aspect="Auto", color_continuous_scale='Portland')
fig.update_layout(
    xaxis_title="Recipient",
    yaxis_title="Passer",
    title="<b>Arsenal 03/04: Pass Matrix (1600mins+ played)<b>",
    )
fig.layout.yaxis.type = 'category'
fig.show()
#py.plot(fig, filename="inv_matrix", auto_open = True)

## Key Passes Donut Chart

In [45]:
pd.options.mode.chained_assignment = None
#get the key pass ids for all henry shots from open play
th14_shots_df = arsenal_shots_df[(arsenal_shots_df.player == "Thierry Henry") & (arsenal_shots_df.shot_type == "Open Play")]
kp_list = th14_shots_df["shot_key_pass_id"].tolist()
th14_kp_df = arsenal_passes_df[arsenal_passes_df['id'].isin(kp_list)]
#rename "id" to "shot_key_pass_id" so it matches up with the shots df
th14_kp_df.rename(columns = {'id':'shot_key_pass_id', 'player':'Key Passer', 'location':'pass_start_location'}, inplace = True)
th14_kp_df = th14_kp_df[['shot_key_pass_id', 'Key Passer', 'pass_length', 'pass_angle', 'pass_start_location', 'pass_end_location', 'pass_cross', 'pass_cut_back', 'pass_switch', 'pass_technique']]
th14_shots_df = th14_shots_df[['shot_key_pass_id', 'player', 'location', 'shot_statsbomb_xg', 'shot_outcome', 'shot_technique']]
th14_kp_df.columns = ['shot_key_pass_id', 'weird', 'Key Passer', 'pass_length', 'pass_angle', 'pass_start_location', 'pass_end_location', 'pass_cross', 'pass_cut_back', 'pass_switch', 'pass_technique']
#append key passes to shots
th14_shots_kp_df = th14_kp_df.merge(th14_shots_df, how='inner')
th14_pivot_df = th14_shots_kp_df.pivot_table(index=['Key Passer'], values='shot_statsbomb_xg', aggfunc=np.sum)
th14_pivot_df.reset_index(inplace=True)
th14_pivot_df.sort_values(by='shot_statsbomb_xg', inplace=True, ascending=False)
#make donut chart showing how many 
fig = px.pie(th14_pivot_df, values='shot_statsbomb_xg', names='Key Passer', hole=0.5, color_discrete_sequence=px.colors.qualitative.Light24, title="03/04: Henry Open Play xG by Key Passer")
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()
#py.plot(fig, filename="allprog25", auto_open = True)

## Pass Direction Polar Charts

### get_angle_counts(pass_df):

In [46]:
def get_angle_counts(pass_df):
    #convert radians to degrees
    pass_df['pass_angle_degrees'] = pass_df['pass_angle'].astype(float).apply(math.degrees)
    pass_df['pass_angle_degrees'] %= 360 
    pass_df.sort_values(by='pass_angle_degrees', inplace=True, ascending=True)
    #12 equally ranged bins
    bins = [-np.inf, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
    labels=[15, 45, 75, 105, 135, 165, 195, 225, 255, 285, 315, 345]
    pass_df["angle_bucket"] = pd.cut(pass_df['pass_angle_degrees'], bins=bins, labels=labels)
    pass_df['angle_bucket'] = pass_df['angle_bucket'].astype("string")
    angles_df = pass_df[['angle_bucket']]
    #print(angles_df)
    angles_df = angles_df.value_counts().to_frame('Count').reset_index()
    #print(angles_df)
    angles_df['angle_bucket'] = angles_df['angle_bucket'].astype(int)

    #add rows if there were zero passes at that angle
    if 15 not in angles_df['angle_bucket'].values:
        row_15 = {'angle_bucket':15, 'Count':0}
        angles_df = angles_df.append(row_15, ignore_index=True)
    if 45 not in angles_df['angle_bucket'].values:
        row_45 = {'angle_bucket':45, 'Count':0}
        angles_df = angles_df.append(row_45, ignore_index=True)
    if 75 not in angles_df['angle_bucket'].values:
        row_75 = {'angle_bucket':75, 'Count':0}
        angles_df = angles_df.append(row_75, ignore_index=True)
    if 105 not in angles_df['angle_bucket'].values:
        row_105 = {'angle_bucket':105, 'Count':0}
        angles_df = angles_df.append(row_105, ignore_index=True)
    if 135 not in angles_df['angle_bucket'].values:
        row_135 = {'angle_bucket':135, 'Count':0}
        angles_df = angles_df.append(row_135, ignore_index=True)
    if 165 not in angles_df['angle_bucket'].values:
        row_165 = {'angle_bucket':165, 'Count':0}
        angles_df = angles_df.append(row_165, ignore_index=True)
    if 195 not in angles_df['angle_bucket'].values:
        row_195 = {'angle_bucket':195, 'Count':0}
        angles_df = angles_df.append(row_195, ignore_index=True)
    if 225 not in angles_df['angle_bucket'].values:
        row_225 = {'angle_bucket':225, 'Count':0}
        angles_df = angles_df.append(row_225, ignore_index=True)
    if 255 not in angles_df['angle_bucket'].values:
        row_255 = {'angle_bucket':255, 'Count':0}
        angles_df = angles_df.append(row_255, ignore_index=True)
    if 285 not in angles_df['angle_bucket'].values:
        row_285 = {'angle_bucket':285, 'Count':0}
        angles_df = angles_df.append(row_285, ignore_index=True)
    if 315 not in angles_df['angle_bucket'].values:
        row_315 = {'angle_bucket':315, 'Count':0}
        angles_df = angles_df.append(row_315, ignore_index=True)
    if 345 not in angles_df['angle_bucket'].values:
        row_345 = {'angle_bucket':345, 'Count':0}
        angles_df = angles_df.append(row_345, ignore_index=True)
    
    #return dataframe with count of buckets
    angles_df.sort_values(by='angle_bucket', inplace=True, ascending=True)
    return angles_df

### Example: Ashley Cole

In [47]:
pd.options.mode.chained_assignment = None

cole_passes_df = arsenal_passes_df[(arsenal_passes_df.player == "Ashley Cole")]
cole_result = get_angle_counts(cole_passes_df)
cole_angles = cole_result["angle_bucket"]
cole_count = cole_result["Count"]

markercolor = ["#0C418C", "#0C418C", "#0C418C", "#F0D339", "#F0D339", "#F0D339", "#56A18F", "#56A18F", "#56A18F", "#DC3912", "#DC3912", "#DC3912"]
hover_data = ["0-30", "30-60", "60-90", "90-120", "120-150", "150-180", "180-210", "210-240", "240-270", "270-300", "300-330", "330-360"]

fig = go.Figure(data=
    go.Barpolar(
        r = cole_count,
        theta = cole_angles,
        width=30, 
        marker_color=markercolor, 
        hovertext=hover_data
    ))

fig.update_layout(
    title="<b>Ashley Cole 03/04: Pass Direction<b>",
    font=dict(
        size=18
    )
    )

#https://medium.com/nerd-for-tech/enriching-data-visualizations-with-annotations-in-plotly-using-python-6127ff6e0f80
fig.add_annotation(
    showarrow=False,
    text="Data Source: <a href='https://statsbomb.com/what-we-do/hub/free-data/'>Statsbomb Open Data</a>",
    x = 0
    , y = -0.15
    , xref='paper'
    , yref='paper' 
    , xanchor='left'
    , yanchor='bottom'
    , xshift=-1
    , yshift=-5
    , font=dict(size=12, color="grey")
    , align="left"
    )

fig.update_layout(showlegend=False)
#fig.update_polars(angularaxis_showticklabels=False)
#fig.update_polars(radialaxis_showticklabels=False)
fig.update_polars(angularaxis_tickfont_size=12)
fig.update_polars(radialaxis_tickfont_size=12)
fig.update_polars(angularaxis_direction="clockwise")
fig.show()
#py.plot(fig, filename="inv_cole_direction", auto_open = True)

### Arsenal 03/04 pass direction maps by position

In [48]:
pd.options.mode.chained_assignment = None

pires_passes_df = arsenal_passes_df[(arsenal_passes_df.player == "Robert Pirès ")]
pires_result = get_angle_counts(pires_passes_df)
pires_angles = pires_result["angle_bucket"]
pires_count = pires_result["Count"]

gilberto_passes_df = arsenal_passes_df[(arsenal_passes_df.player == "Gilberto Aparecido da Silva")]
gilberto_result = get_angle_counts(gilberto_passes_df)
gilberto_angles = gilberto_result["angle_bucket"]
gilberto_count = gilberto_result["Count"]

vieira_passes_df = arsenal_passes_df[(arsenal_passes_df.player == "Patrick Vieira")]
vieira_result = get_angle_counts(vieira_passes_df)
vieira_angles = vieira_result["angle_bucket"]
vieira_count = vieira_result["Count"]

ljungberg_passes_df = arsenal_passes_df[(arsenal_passes_df.player == "Fredrik Ljungberg")]
ljungberg_result = get_angle_counts(ljungberg_passes_df)
ljungberg_angles = ljungberg_result["angle_bucket"]
ljungberg_count = ljungberg_result["Count"]

In [49]:
markercolor = ["#0C418C", "#0C418C", "#0C418C", "#F0D339", "#F0D339", "#F0D339", "#56A18F", "#56A18F", "#56A18F", "#DC3912", "#DC3912", "#DC3912"]

fig = make_subplots(rows=1, cols=4, subplot_titles=("Pirès", "Gilberto", "Vieira", "Ljungberg"), specs=[[{"type": "polar"}, {"type": "polar"}, {"type": "polar"}, {"type": "polar"}]])

fig.add_trace(go.Barpolar(r=pires_count, theta=pires_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=1)

fig.add_trace(go.Barpolar(r=gilberto_count, theta=gilberto_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=2)

fig.add_trace(go.Barpolar(r=vieira_count, theta=vieira_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=3)

fig.add_trace(go.Barpolar(r=ljungberg_count, theta=ljungberg_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=4)

fig.update_layout(
    title="<b>Arsenal Midfield 03/04: Pass Direction<b>",
    font=dict(
        size=18
    )
    )

#https://medium.com/nerd-for-tech/enriching-data-visualizations-with-annotations-in-plotly-using-python-6127ff6e0f80
fig.add_annotation(
    showarrow=False,
    text="Data Source: <a href='https://statsbomb.com/what-we-do/hub/free-data/'>Statsbomb Open Data</a>",
    x = 0
    , y = -0.15
    , xref='paper'
    , yref='paper' 
    , xanchor='left'
    , yanchor='bottom'
    , xshift=-1
    , yshift=-5
    , font=dict(size=12, color="grey")
    , align="left"
    )

fig.update_layout(showlegend=False)
fig.update_polars(angularaxis_showticklabels=False)
fig.update_polars(radialaxis_showticklabels=False)
fig.update_polars(angularaxis_direction="clockwise")
fig.show()
#py.plot(fig, filename="inv_mid_direction", auto_open = True)

In [50]:
pd.options.mode.chained_assignment = None

th14_rec_df = arsenal_passes_df[(arsenal_passes_df.pass_recipient == "Thierry Henry")]
pires_2_henry_df = th14_rec_df[(th14_rec_df.player == "Robert Pirès ")]
cole_2_henry_df = th14_rec_df[(th14_rec_df.player == "Ashley Cole")]
vieira_2_henry_df = th14_rec_df[(th14_rec_df.player == "Patrick Vieira")]

pires_result = get_angle_counts(pires_2_henry_df)
pires_angles = pires_result["angle_bucket"]
pires_count = pires_result["Count"]

cole_result = get_angle_counts(cole_2_henry_df)
cole_angles = cole_result["angle_bucket"]
cole_count = cole_result["Count"]

vieira_result = get_angle_counts(vieira_2_henry_df)
vieira_angles = vieira_result["angle_bucket"]
vieira_count = vieira_result["Count"]

In [51]:
markercolor = ["#0C418C", "#0C418C", "#0C418C", "#F0D339", "#F0D339", "#F0D339", "#56A18F", "#56A18F", "#56A18F", "#DC3912", "#DC3912", "#DC3912"]

fig = make_subplots(rows=1, cols=3, subplot_titles=("Pirès", "Vieira", "Cole"), specs=[[{"type": "polar"}, {"type": "polar"}, {"type": "polar"}]])

fig.add_trace(go.Barpolar(r=pires_count, theta=pires_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=1)

fig.add_trace(go.Barpolar(r=vieira_count, theta=vieira_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=2)

fig.add_trace(go.Barpolar(r=cole_count, theta=cole_angles, width=30, marker_color=markercolor, hovertext=hover_data),
    row=1, col=3)

fig.update_layout(
    title="<b>Arsenal 03/04: Pass Direction -> Henry<b>",
    font=dict(
        size=18
    )
    )

#https://medium.com/nerd-for-tech/enriching-data-visualizations-with-annotations-in-plotly-using-python-6127ff6e0f80
fig.add_annotation(
    showarrow=False,
    text="Data Source: <a href='https://statsbomb.com/what-we-do/hub/free-data/'>Statsbomb Open Data</a>",
    x = 0
    , y = -0.15
    , xref='paper'
    , yref='paper' 
    , xanchor='left'
    , yanchor='bottom'
    , xshift=-1
    , yshift=-5
    , font=dict(size=12, color="grey")
    , align="left"
    )

fig.update_layout(showlegend=False)
fig.update_polars(angularaxis_showticklabels=False)
fig.update_polars(radialaxis_showticklabels=False)
fig.update_polars(angularaxis_direction="clockwise")
fig.show()
#py.plot(fig, filename="direction_2_henry", auto_open = True)